In [1]:
# | default_exp tools.execute_bash

In [1]:
# | export
from agentic.tools.base import BaseTool, ToolMetadata, ToolCategory, create_success_response, create_error_response, extract_validation_error, create_success_response, create_error_response

import subprocess
import os
from typing import Dict, Any, Optional
from pydantic import BaseModel, ValidationError
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

class ExecuteBashParams(BaseModel):
    command: str
    working_dir: Optional[str] = None
    timeout: Optional[int] = 60
    env_vars: Optional[Dict[str, str]] = None
    capture_output: Optional[bool] = True
    shell: Optional[str] = "/bin/bash"
    summary: Optional[str] = None

class ExecuteBashTool(BaseTool):
    def __init__(self):
        metadata = ToolMetadata(
            name="execute_bash",
            description="Execute bash commands with safety controls",
            category=ToolCategory.SYSTEM,
            requires_approval=False,  # Disabled for automated testing
            is_dangerous=False        # Disabled for automated testing
        )
        super().__init__(metadata)

    def get_parameters_schema(self) -> Dict[str, Any]:
        try:
            return {
                "type": "object",
                "properties": {
                    "command": {"type": "string", "description": "Bash command to execute"},
                    "working_dir": {"type": "string", "description": "Working directory for command execution"},
                    "timeout": {"type": "integer", "description": "Timeout in seconds (default: 30)"},
                    "env_vars": {"type": "object", "description": "Additional environment variables"},
                    "capture_output": {"type": "boolean", "description": "Capture stdout/stderr (default: true)"},
                    "shell": {"type": "string", "description": "Shell to use (default: /bin/bash)"},
                    "summary": {"type": "string", "description": "Brief description of what the command does"}
                },
                "required": ["command"]
            }
        
        except Exception as e:
            logger.error(f"Schema generation failed: {e}")
            return {}

    def _check_permissions(self, command: str, working_dir: Optional[str] = None) -> bool:
        """Check if command requires elevated permissions and prompt user"""
        
        # Commands that typically require sudo
        sudo_commands = ['sudo', 'apt', 'yum', 'dnf', 'systemctl', 'service', 'mount', 'umount', 'chmod +x', 'chown']
        
        # Check if command needs sudo
        needs_sudo = any(cmd in command.lower() for cmd in sudo_commands)
        
        # Check if writing to system directories
        system_dirs = ['/usr', '/etc', '/var', '/opt', '/root']
        if working_dir and any(working_dir.startswith(sdir) for sdir in system_dirs):
            needs_sudo = True
            
        # Check if command writes to system locations
        if any(f'> {sdir}' in command or f'>> {sdir}' in command for sdir in system_dirs):
            needs_sudo = True
            
        if needs_sudo:
            print(f"⚠️  Command may require elevated permissions: {command}")
            response = input("Do you want to proceed? (y/N): ").strip().lower()
            return response in ['y', 'yes']
            
        return True
    
    def _handle_permission_error(self, command: str, error: str) -> Dict[str, Any]:
        """Handle permission denied errors by suggesting solutions"""
        
        if "permission denied" in error.lower() or "operation not permitted" in error.lower():
            print(f"❌ Permission denied for command: {command}")
            print("💡 Possible solutions:")
            print("   1. Run with sudo (if system command)")
            print("   2. Check file/directory permissions")
            print("   3. Ensure you own the target files/directories")
            
            response = input("Try with sudo? (y/N): ").strip().lower()
            if response in ['y', 'yes']:
                sudo_command = f"sudo {command}"
                print(f"🔄 Retrying with: {sudo_command}")
                return {"retry_command": sudo_command}
                
        return {"no_retry": True}

    def execute(self, **kwargs) -> Dict[str, Any]:
        try:
            params = ExecuteBashParams(**kwargs)
        except ValidationError as e:
            return create_error_response(f"Invalid parameters: {extract_validation_error(e)}")
        except Exception as e:
            return create_error_response(f"Parameter validation failed: {str(e)}")
        
        try:
            # Check permissions before execution
            if not self._check_permissions(params.command, params.working_dir):
                return create_error_response("Command execution cancelled by user due to permission requirements")
            
            shell = params.shell or "/bin/bash"
            
            # First attempt
            result = subprocess.run(
                params.command,
                shell=True,
                capture_output=params.capture_output,
                text=True,
                timeout=params.timeout,
                cwd=params.working_dir,
                env={**os.environ, **(params.env_vars or {})} if params.env_vars else None
            )
            
            # Check for permission errors and offer retry
            if result.returncode != 0 and result.stderr:
                retry_info = self._handle_permission_error(params.command, result.stderr)
                
                if "retry_command" in retry_info:
                    # Retry with sudo
                    retry_result = subprocess.run(
                        retry_info["retry_command"],
                        shell=True,
                        capture_output=params.capture_output,
                        text=True,
                        timeout=params.timeout,
                        cwd=params.working_dir,
                        env={**os.environ, **(params.env_vars or {})} if params.env_vars else None
                    )
                    
                    if retry_result.returncode == 0:
                        return create_success_response(
                            f"Command executed successfully with elevated privileges",
                            data={
                                "stdout": retry_result.stdout,
                                "stderr": retry_result.stderr,
                                "exit_status": retry_result.returncode,
                                "command": retry_info['retry_command'],
                                "retried_with_sudo": True
                            }
                        )
                    else:
                        return create_error_response(
                            f"Command failed even with elevated privileges (exit code {retry_result.returncode})",
                            data={
                                "stdout": retry_result.stdout,
                                "stderr": retry_result.stderr,
                                "exit_status": retry_result.returncode,
                                "command": retry_info['retry_command'],
                                "retried_with_sudo": True
                            }
                        )
            
            if result.returncode == 0:
                return create_success_response(
                    f"Command executed successfully",
                    data={
                        "stdout": result.stdout,
                        "stderr": result.stderr,
                        "exit_status": result.returncode,
                        "command": params.command
                    }
                )
            else:
                return create_error_response(
                    f"Command failed with exit code {result.returncode}",
                    data={
                        "stdout": result.stdout,
                        "stderr": result.stderr,
                        "exit_status": result.returncode,
                        "command": params.command
                    }
                )
            
        except subprocess.TimeoutExpired as e:
            return create_error_response(
                f"Command timed out after {params.timeout} seconds",
                data={"command": params.command, "timeout": params.timeout}
            )
        except subprocess.CalledProcessError as e:
            return create_error_response(
                f"Command execution failed: {str(e)}",
                data={
                    "stdout": getattr(e, 'stdout', ''),
                    "stderr": getattr(e, 'stderr', ''),
                    "exit_status": e.returncode,
                    "command": params.command
                }
            )
        except Exception as e:
            return create_error_response(f"Unexpected execution error: {str(e)}")



ImportError: cannot import name 'create_success_response' from 'agentic.tools.base' (/home/pranav-pc/projects/applied-GenAI-lab/agentic/tools/base.py)